In [ ]:
#! brew install boost-python
! pip install box2d
! pip install gym
! pip install tqdm

In [ ]:
#! pip install git+https://github.com/openai/gym.git#egg=gym[box2d]

In [1]:
import numpy as np
import random
import torch 
import torch.nn as nn
import gym

from tqdm.notebook import trange, tqdm

from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
#import gym
from gym import wrappers

env = gym.make('LunarLander-v2')
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done: break
env.close()

Failed attempt to visualize

```
import gym
from gym import wrappers

env = gym.make('LunarLander-v2')
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done: break
env.close()
```

Another failed attempt
```
from IPython import display
import os
env = gym.make('LunarLander-v2')

#games_obs = env.reset()

#ani = create_animation(render_frames(env, 300))
plt.imshow(env.render('rgb_array'))
plt.grid(False)
```


clear_output()

from xvfbwrapper import Xvfb
vdisplay = Xvfb(width=1280, height=740)
vdisplay.start()

rcParams['figure.figsize'] = 5, 3

def render_frames(env, num_frame=50):
    
    env.reset()
    frames = []
    for i in range(num_frame):
        
        _, _, done, _ = env.step( env.action_space.sample())
        
        if done:
            env.reset()
            
        frames.append( env.render(mode="rgb_array"))
    return frames

def create_animation(frames):
    rc('animation', html='jshtml')
    fig = plt.figure()
    plt.axis('off')
    im = plt.imshow(frames[0], animated=True)
    
    def update_fig(i):
        im.set_array(frames[i])
        return im
    
    ani = animation.FuncAnimation(fig, 
                                  update_fig, 
                                  frames=len(frames), 
                                  interval=len(frames)/10, 
                                  blit=True)
    display(HTML(ani.to_html5_video()))
    plt.close()
    
    return ani

import gym
!pip install box2d
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

!python -m pip install pyvirtualdisplay
#from pyvirtualdisplay import Display
#display = Display(visible=0, size=(1400, 900))
#display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [3]:
import gym

env = gym.make("LunarLander-v2")
env.reset()

for _ in range(1000):
    env.render()
    _, _, is_done, _ = env.step(env.action_space.sample()) #take a random action
    
    if is_done:
        break

env.close()

In [4]:
class ValueFunction(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        
        self.V = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.SELU(),
            nn.Linear(128, 256),
            nn.SELU(),
            nn.Linear(256, 1),
        )
        
    def forward(self, x):
        v = self.V(x)

        return v

In [5]:
class SoftmaxPolicy(nn.Module):
    def __init__(self, input_size, num_actions):
        super().__init__()
        
        self.input_size = input_size
        self.num_actions = num_actions
        
        self.pi = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.SELU(),
            nn.Linear(128, 256),
            nn.SELU(),
            nn.Linear(256, 256),
            nn.SELU(),
            nn.Linear(256, num_actions),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        logits = self.pi(x)
        
        return logits

In [6]:
cc = 0 

In [7]:
from torch.distributions.categorical import Categorical
import itertools

obs = env.reset()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pi = SoftmaxPolicy(input_size=8, num_actions=4).to(device)
old_pi = SoftmaxPolicy(input_size=8, num_actions=4).to(device)
value_fun = ValueFunction(input_size=8).to(device)

state = torch.from_numpy(obs).float()
logits = pi(state.unsqueeze(0).to(device))
logits.squeeze(0)

sample_action(logits.squeeze(0).cpu())


NameError: name 'sample_action' is not defined

In [8]:
γ = 0.99

def sample_action(logits):
    return Categorical(logits=logits).sample()

def collect_experience(env, policy):
    obs = env.reset()
    
    memory = []
    rewards = []
    states = []
    actions = []
    logps = []
    
    policy.eval()
    
    with torch.no_grad():
        for i in itertools.count():
            state = torch.from_numpy(obs).float()
            logits = policy(state.unsqueeze(0).to(device))
            act = sample_action(logits.cpu().squeeze())
            
            obs, reward, is_done, onfi = env.step(act.item())
            
            states.append(state)
            actions.append(act)
            rewards.append(torch.tensor(reward/200.0, dtype=torch.float))
            logps.append(logits[0, act.item()])
            
            if is_done:
                break
                
    values = []
    acc = 0.0
    for r in reversed(rewards):
        acc = γ * acc + r
        values.insert(0, acc)
    
    return zip(states, actions, values, logps), np.sum(rewards)

In [9]:
import math
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_value_

params = itertools.chain(pi.parameters(), value_fun.parameters())
optim = torch.optim.Adam(params, lr=1e-4)


In [10]:
from collections import deque

eps = 0.2
step = 0
rewards = []
scores_window = deque(maxlen=100)
for loop in trange(800):
    
    rewards.append(0)
    experiences = []
    

    for episode in range(30):
        exp, reward = collect_experience(env, pi)
        experiences.extend( exp )
        rewards[loop] += reward

    scores_window.append(rewards[loop])
            
    dataloader = DataLoader(experiences, batch_size=32, shuffle=True)
    
    value_fun.train()
    pi.train()
    
    old_pi.load_state_dict( pi.state_dict())
    old_pi.eval()

    for state, action, value, logp_old in dataloader:
        step += 1
        state_on_device = state.to(device)
        state_value = value_fun(state_on_device)
        value_loss = nn.functional.mse_loss(input=state_value, 
                                     target=value.unsqueeze(1).to(device))
        
        logits = pi(state_on_device)
        logp = logits.gather(dim=1, index=action.unsqueeze(1).to(device))
        
        with torch.no_grad():
            advantage = value.unsqueeze(1).to(device) - state_value
            
        ratio = (logp - logp_old.unsqueeze(1).to(device)).exp()
        entropy = - logits * logits.exp()
        
        pi_loss = - torch.min( ratio * advantage, 
                             torch.clamp(ratio, min=1.0 - eps, max = 1.0 + eps) * advantage) - 1e-4 * entropy
        pi_loss = pi_loss.mean()
        
        loss = pi_loss + value_loss
        
        optim.zero_grad()
        loss.backward()
        
        clip_grad_value_( params, 1.0)
        optim.step()
    if loop % 25 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(loop, np.mean(scores_window)))
        
     

Episode 0	Average Score: -33.78
Episode 25	Average Score: -14.08
Episode 50	Average Score: -2.66
Episode 75	Average Score: 8.08
Episode 100	Average Score: 15.80
Episode 125	Average Score: 27.85
Episode 150	Average Score: 35.18
Episode 175	Average Score: 37.33
Episode 200	Average Score: 37.94
Episode 225	Average Score: 39.18
Episode 250	Average Score: 39.70
Episode 275	Average Score: 39.97
Episode 300	Average Score: 40.15
Episode 325	Average Score: 40.12
Episode 350	Average Score: 39.71
Episode 375	Average Score: 39.76
Episode 400	Average Score: 39.61
Episode 425	Average Score: 39.88
Episode 450	Average Score: 40.40
Episode 475	Average Score: 40.71
Episode 500	Average Score: 40.95
Episode 525	Average Score: 40.91
Episode 550	Average Score: 40.81
Episode 575	Average Score: 40.71
Episode 600	Average Score: 40.66
Episode 625	Average Score: 40.64
Episode 650	Average Score: 39.90
Episode 675	Average Score: 39.75
Episode 700	Average Score: 39.70
Episode 725	Average Score: 39.68
Episode 750	Av

In [13]:

for _ in range(20):
    
    obs = env.reset()
        
    with torch.no_grad():
        for i in range(1000):
            env.render()
            state = torch.from_numpy(obs).float()
            logits = pi(state.unsqueeze(0).to(device))
            act = sample_action(logits.cpu().squeeze())
            
            obs, _, is_done, _ = env.step(act.item()) #take a random action

            if is_done:
                break

env.close()